In [1]:
import random
import numpy as np
import random
import pickle
import letor_metrics
import pyximport
import sys
from tqdm import tqdm
pyximport.install()
import matplotlib


raw_data_train = np.loadtxt('/data/recnet_draft/ml100k/train_all_raw.csv', skiprows = 1, delimiter=',')
raw_data_test = np.loadtxt('/data/recnet_draft/ml100k/test_all_raw.csv', skiprows = 1, delimiter=',')
raw_data = np.concatenate((raw_data_train, raw_data_test))
from dataset_tt_static import TripletsDataset

ds = TripletsDataset(raw_data_train, raw_data_test, threshold_user=60, rnd_seed=42)
ds.train_test_split()

ds.init_cached_random()
import tensorflow as tf
import bprnn_ml100k_param_tune_all
import imp

N_USERS = int(max(raw_data[:, 0])) + 1
N_ITEMS = int(max(raw_data[:, 1])) + 1
N_EMBEDDINGS = 2

import tensorflow.contrib.slim as slim
imp.reload(bprnn_ml100k_param_tune_all)

#%%
def inner_network(user_emb, item_emb):
    joined_input = tf.concat([user_emb, item_emb], 1)
    net = slim.fully_connected(inputs=joined_input, num_outputs=16, activation_fn=tf.nn.relu)
#     net = slim.fully_connected(inputs=joined_input, num_outputs=64, activation_fn=tf.nn.relu)
#     net = slim.dro
    net = slim.fully_connected(inputs=net, num_outputs=1, activation_fn=None)
    return net

model = bprnn_ml100k_param_tune_all.BPR_NN(N_USERS, N_ITEMS, N_EMBEDDINGS, alpha=0, beta=1, alpha_reg=0.005, inner_net=inner_network)
model.build_graph()
model.initialize_session()

losses = []
batch_size = 512
for n_batches, cur_optim in [(10000, model.trainer_3)]:
    for i in tqdm(range(n_batches)):
        if i%1000==0 and i/1000 >= 1:
            model.alpha = model.alpha + 0.1
            model.beta = model.beta - 0.1
            print('batch number:'+str(i))
            print('alpha:'+ str(model.alpha))
            print('beta:'+ str(model.beta))
        batch = ds.sample_train_batch(n_samples=batch_size)
        fd = {
            model.user_ids:  batch['users'], 
            model.left_ids:  batch['left_items'],
            model.right_ids: batch['right_items'],
            model.target_y:  batch['y'],
        }
        el, nl, reg, t, _ = model.session.run(
            [model.embedding_loss, model.net_loss, model.regularization, model.target, cur_optim], 
            feed_dict=fd
        )
        losses.append((el, nl, reg, t))
        if i%1000==0:
            user_norm = np.linalg.norm(model.weights_u)
            item_norm = np.linalg.norm(model.weights_i)
            print('[it {}] weight norms, users: {}, items: {}'.format(i, user_norm, item_norm))
            print('[it {}] metrics (emb_loss, net_loss, reg, target): {}'.format(i, losses[-1]))

Split users:   9%|▉         | 38/408 [00:00<00:02, 171.85it/s]

  0%|          | 16/10000 [00:00<20:42,  8.04it/s]

[it 0] weight norms, users: 24.89217185974121, items: 33.103614807128906
[it 0] metrics (emb_loss, net_loss, reg, target): (0.70632982, 0.69421673, 1.9343712, 0.71600169)


 10%|█         | 1018/10000 [00:06<00:56, 158.28it/s]

batch number:1000
alpha:0.1
beta:0.9
[it 1000] weight norms, users: 27.076431274414062, items: 32.525390625
[it 1000] metrics (emb_loss, net_loss, reg, target): (0.58758307, 0.68635154, 2.5332975, 0.60024953)


...

 80%|████████  | 8017/10000 [00:58<00:14, 139.59it/s]

batch number:8000
alpha:0.7999999999999999
beta:0.20000000000000015
[it 8000] weight norms, users: 38.699920654296875, items: 44.10477828979492
[it 8000] metrics (emb_loss, net_loss, reg, target): (0.45737207, 0.68202066, 5.3895392, 0.48431978)


 90%|█████████ | 9033/10000 [01:05<00:06, 159.97it/s]

batch number:9000
alpha:0.8999999999999999
beta:0.10000000000000014
[it 9000] weight norms, users: 39.09944152832031, items: 45.299373626708984
[it 9000] metrics (emb_loss, net_loss, reg, target): (0.44907317, 0.69325233, 5.701674, 0.47758153)


100%|██████████| 10000/10000 [01:11<00:00, 139.37it/s]


In [3]:
export_basename = '/data/recnet_draft/ml100k/inc_alpha/vectors/'
export_pred = open(export_basename + 'pr_ml100k_10', 'w')
export_true = open(export_basename + 'gt_ml100k_10', 'w')

ndcg_vals = []
for u in tqdm(ds.data_keys, desc='Prediction', leave=True):
    # if not u in ds.test:
    if not u in ds.test or not ds.test[u]:
        continue
    response = np.zeros(len(ds.test[u]))
    fd = {
            model.user_ids:  (np.ones(len(ds.test[u]))*u).astype(np.int32), 
            model.left_ids:  np.array([i for (i, r) in ds.test[u]]).astype(np.int32),
        }
    response += model.session.run(model.embedding_left, feed_dict=fd)[:, 0]
    response += model.session.run(model.left_output, feed_dict=fd)[:, 0]

    # make relevances
    relevances = np.array([r for (i, r) in ds.test[u]])
    items = np.array([i for (i, r) in ds.test[u]])  # it's already sorted by true relevance
    itemsGroundTruth = np.array([i for (i,r) in ds.test[u] if r == 1])
    predicted_ranking = np.argsort(-response)

    # write down predictions
    export_pred.write(' '.join(map(str, [u] + list(items[predicted_ranking]))) + '\n')
    export_true.write(' '.join(map(str, [u] + list(itemsGroundTruth))) + '\n')

    # calc score
    gain = letor_metrics.ndcg_from_ranking(relevances, predicted_ranking, 10)
    ndcg_vals.append(gain)


# In[32]:

print(ndcg_vals)

Prediction: 100%|██████████| 408/408 [00:00<00:00, 3532.60it/s]

[0.87564587272673344, 0.5730249097309823, 0.48993086854170803, nan, 1.0, 0.8896955117809705, 0.69693835499843326, 0.8611375561264456, 0.63092975357145753, 0.57064171895532012, 0.9284591831945489, 1.0, 0.0, 0.33333333333333331, 0.53418194731807067, 0.70580751486785287, 0.63092975357145753, 1.0, 0.53072127397724345, 0.76063956823570356, 0.90602543553468229, 0.51006913145829202, 0.84551932527972595, 0.77561996045262482, 0.59123520482302772, 0.88545988157148736, 0.62818704803510339, nan, 1.0, 0.99634880215493538, 0.07336392209936006, 0.85123609415942747, 1.0, 0.79358406776491097, 1.0, 0.97474295285678791, 1.0, 0.80481099920933907, 0.61410696267909359, 1.0, 1.0, 0.92493723314230458, 1.0, 0.29155904987671011, nan, 0.98483309595535218, nan, nan, 0.63092975357145753, 0.69947390308139201, 0.92663607790063995, 0.5184650094568698, 1.0, 1.0, 0.38685280723454163, 0.91972078914818756, 1.0, 0.57064171895532012, 1.0, 1.0, 0.94415922634006066, nan, nan, 1.0, 0.96746798348916929, 0.88995411685096004, 0.

In [ ]:
from matplotlib.pyplot import *

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows = 3, ncols = 2)
#ax[-1, -1].axis('off')

for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[0] for x in losses[:int]], c='b', label='embedding_loss')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('embedding loss')
    yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/embedding_loss_dec_alpha.pdf')
plt.close()

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows=3, ncols=2)

for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[1] for x in losses[:int]], c='m', label='ranking_loss')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('logloss')
    yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/ranking_loss_dec_alpha.pdf')
plt.close()

In [ ]:
import matplotlib.pyplot as plt
ints = [100, 500, 1000, 2000, 5000, 10000]
pos = 1
fig, ax = plt.subplots(nrows=3, ncols=2)
for int in ints:
    axis = plt.subplot(3, 2, pos)
    axis.plot([x[3] for x in losses[: int]], c='g', label='target')
    grid()
    legend()
    xlabel('n_batches')
    ylabel('recnet loss')
    #yscale('log')
    draw()
    show(block=False)
    pos = pos + 1
    fig.savefig('/home/sumit/recnet_draft/plots/ml100k/target_loss_dec_alpha')
plt.close()
    

In [ ]:
plt.close("all")